이번에 구현해볼 예제는 다음과 같이 키워드를 정의하고, 이를 추출하는 함수입니다. 키워드는 관점이 주어졌을 때, 그 관점에서 더 자주 등장하는 단어로 정의할 수 있습니다. 예를 들어 여름 철 평상시에 뉴스에서 ‘폭우’가 0.1% 등장하는데, 오늘의 뉴스에서 ‘폭우’가 1% 등장하였다면, ‘폭우’는 오늘 뉴스의 키워드입니다. 이를 다음과 같이 공식화 할 수 있습니다. 

    score(w) = P(w|Dt) / { P(w|Dt) + P(w|Dr) }

    P(w|Dt): target document에서 단어 w가 출현한 비율
    P(w|Dr): reference document에서 단어 w가 출현한 비율



2016-10-20 일 뉴스에 명사 추출기를 이용하여 명사만을 걸러낸 document - term matrix 를 이용합니다.

In [1]:
import config
from navernews_10days import get_bow

x, _idx_to_vocab, _vocab_to_idx = get_bow(tokenize='noun', date='2016-10-20')

soynlp=0.0.49
added lovit_textmining_dataset


In [2]:
def vocab_to_idx(word):
    return _vocab_to_idx.get(word, -1)

def idx_to_vocab(idx):
    if 0 <= idx < len(_idx_to_vocab):
        return _idx_to_vocab[idx]
    return None

vocab_to_idx('아이오아이')

5537

'아이오아이'라는 단어가 포함된 문서 집합을 positive_documents로, 그렇지 않은 문서 집합을 negative_documents로 둡니다. 

In [3]:
import numpy as np

word = '아이오아이'
word_idx = vocab_to_idx(word)
positive_documents = x[:,word_idx].nonzero()[0]
negative_documents = np.asarray(
    [i for i in range(x.shape[0]) if not (i in positive_documents)]
)

print('n pos = {}, n neg = {}'.format(len(positive_documents), len(negative_documents)))

n pos = 97, n neg = 29994


sparse matrix x에서 sum()을 하면 모든 값의 합이 구해집니다. sum(axis=0)을 하면 rows가 하나의 row로 합쳐지는 sum이며, sum(axis=1)을 하면 columns가 하나의 column으로 합쳐지는 sum입니다. 우리의 x는 (document by term) matrix이기 때문에 row sum을 하면 모든 문서에서의 단어들의 빈도수 합이 구해집니다. 그래서 (30091 by 9774)의 term frequency matrix가 9774 차원의 term frequency vector가 되었음을 확인할 수 있습니다. 

In [4]:
x.shape

(30091, 9774)

In [5]:
x.sum(axis=0).shape

(1, 9774)

scipy.sparse 의 matrix는 slicing이 가능합니다. positive_documents를 list 형식으로 만들었습니다. 이 list를 x에 넣어서 x[list,:] 을 실행하면 list에 해당하는 모든 row들을 잘라서 submatrix를 만듭니다. 

In [6]:
positive_documents[:30]

array([ 6884,  6897,  6956,  7338,  7345,  7582,  8011,  8053,  9180,
        9228,  9494,  9539,  9876,  9894, 13059, 13231, 13691, 13856,
       14117, 15573, 15836, 15868, 15880, 16198, 16485, 16487, 16489,
       16490, 16492, 17304], dtype=int32)

positive_documents, 즉 '아이오아이'라는 단어가 들어간 문서들만을 잘라내어 submatrix를 만든 뒤, 이를 row sum (= sum(axis=0))을 하였습니다. '아이오아이'라는 단어가 들어간 문서의 단어 빈도수가 만들어집니다. 

    positive_proportion = x[positive_documents,:].sum(axis=0)

이를 list로 만든 뒤, 출력해보면 다음과 같이 term frequency list가 만들어졌음을 볼 수 있습니다. 길이는 단어의 개수와 같습니다. 

In [7]:
positive_proportion = np.asarray(x[positive_documents,:].sum(axis=0))[0]
positive_proportion.shape

(9774,)

총 합을 \_sum 이라는 변수로 만든 뒤, 모든 빈도수를 이 \_sum으로 나누어주면 positive documents, 즉 '아이오아이'가 포함된 문서에서의 단어들의 출현 비율이 만들어집니다. 

In [8]:
positive_proportion = positive_proportion / positive_proportion.sum()
positive_proportion[:30]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00010782, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.00010782, 0.00021563,
       0.        , 0.00032345, 0.        , 0.00204852, 0.00010782,
       0.        , 0.        , 0.        , 0.        , 0.        ])

이 과정을 반복할 것이니 to_proportion(documents_list) 라는 함수로 만들어 둡니다. 

positive proportion은 '아이오아이'가 포함된 문서에서의 단어 출현 비율, negative proportion은 '아이오아이'가 포함되지 않은 문서에서의 단어 출현 비율입니다. 

In [9]:
def to_proportion(documents_list):
    proportion = np.asarray(x[documents_list,:].sum(axis=0))[0]
    proportion = proportion / proportion.sum()
    return proportion

positive_proportion = to_proportion(positive_documents)
negative_proportion = to_proportion(negative_documents)

상대적 출현 비율은 모든 단어들에 대하여 p / (p+n) 을 계산하면 됩니다. p는 한 단어의 positive proportion의 값이며, n은 그 단어의 negative proportion의 값입니다. 

In [10]:
def proportion_ratio(pos, neg):
    assert len(pos) == len(neg)
    ratio = pos / (pos + neg)
    ratio = np.nan_to_num(ratio)
    return ratio

keyword_score = proportion_ratio(positive_proportion, negative_proportion)

In [11]:
keyword_score[:30]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.8844969 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.54934511, 0.56280303,
       0.        , 0.82732507, 0.        , 0.87481523, 0.54733158,
       0.        , 0.        , 0.        , 0.        , 0.        ])

이제 proportion ratio가 높은 단어들을 찾아봅니다. enumerate를 이용하면 점수가 높은 단어의 index와 그 점수를 (단어, 점수) pair로 만들 수 있습니다. 

    enumerate(keyword_score)

이를 점수 기준으로 정렬하면 점수 순 정렬이 됩니다. 

    sorted(enumerate(keyword_score), key=lambda x:-x[1])

In [12]:
sorted(enumerate(keyword_score), key=lambda x:-x[1])[:30]

[(4309, 1.0),
 (5537, 1.0),
 (2308, 0.9999606090273322),
 (5333, 0.998991194480233),
 (6145, 0.9989863725521622),
 (921, 0.9982710816259988),
 (2466, 0.9981432513884275),
 (5880, 0.9978307775631691),
 (4815, 0.9978210421997837),
 (3682, 0.9975836317984187),
 (6208, 0.9973594469004617),
 (4701, 0.9963128215975839),
 (4441, 0.9958319893090414),
 (7832, 0.9948644479894773),
 (9113, 0.9946890725030576),
 (6037, 0.9938200380735884),
 (8976, 0.9934422266805437),
 (7126, 0.9929667382454291),
 (4546, 0.9909673401797572),
 (4859, 0.9908127932033489),
 (5879, 0.9907514986652862),
 (1103, 0.99017203825805),
 (6904, 0.9884164745297143),
 (6584, 0.9881439461828352),
 (4343, 0.9880894585465715),
 (4700, 0.9875086307696628),
 (8721, 0.9869906112674688),
 (8651, 0.9867835556082788),
 (4035, 0.98596911773225),
 (2120, 0.9853881990008125)]

앞서 term frequency vector를 만들었습니다. 이도 list로 만들어 둡니다. 키워드/연관어를 추출할 때, 최소 빈도수를 설정하기 위해서입니다. 

In [13]:
term_frequency = np.asarray(x.sum(axis=0))[0]
term_frequency[:30]

array([ 462,  769,   48,   59,   54,  332,  499,   52,   40, 2683,  195,
         40,   49,  144,   55,  323,   45,  222,  246,  466,  289,  190,
         37,  831,  248,   91,   78,   53,   57,   49], dtype=int64)

이 과정을 proportion ratio keyword로 감싸서 함수로 만들어 둡니다. min count와 단어를 입력받도록 합니다. 

term frequency matrix 에 포함되지 않은 단어면 키워드분석을 하지 않습니다. 

    word_idx = word2int(word)
        if word_idx == -1:
            return None
            
min count cutting을 통해서 최소 빈도수 이상인 단어들만 available terms로 만들어 둡니다. 

    term_frequency = x.sum(axis=0).tolist()[0]
    available_terms = {term:count for term, count in enumerate(term_frequency) if count >= min_count}
    
그 뒤 positive_documents / negative_documents를 선택하고, positive_proportion / negative_proportion 를 계산한 뒤, proportion_ratio를 계산합니다. 

    positive_documents = x[:,word_idx].nonzero()[0].tolist()
    positive_proportion = to_proportion(positive_documents)
    ...
    keyword_score = proportion_ratio(positive_proportion, negative_proportion)
    
최소빈도수 이상으로 등장한 단어만을 keyword로 남겨두는 filtering을 합니다. filter 함수를 써도 좋습니다.

    keyword_score = [(term, score) for term, score in keyword_score if term in available_terms]

word index로 표현되어 있는 keyword_score = [(idx, score), ... ]를 [(word, score), ...]로 바꿔줍니다. 

    keyword_score = [(int2word(term), score) for term, score in keyword_score]

In [14]:
def proportion_ratio_keyword(word, x, min_count=10):
    idx = vocab_to_idx(word)
    if idx == -1:
        return None
    
    term_frequency = np.asarray(x.sum(axis=0))[0]
    available_terms = set(np.where(term_frequency >= min_count)[0])

    positive_documents = x[:,idx].nonzero()[0]
    negative_documents = np.asarray([i for i in range(x.shape[0]) if not (i in positive_documents)])

    positive_proportion = to_proportion(positive_documents)
    negative_proportion = to_proportion(negative_documents)
    
    keyword_score = proportion_ratio(positive_proportion, negative_proportion)
    keyword_score = sorted(enumerate(keyword_score), key=lambda x:-x[1])
    keyword_score = [(term, score) for term, score in keyword_score if term in available_terms]
    keyword_score = [(idx_to_vocab(term), score) for term, score in keyword_score]

    return keyword_score

In [15]:
from pprint import pprint

keywords = proportion_ratio_keyword('아이오아이', x, min_count=30)
pprint(keywords[:30])

[('빅브레인', 1.0),
 ('아이오아이', 1.0),
 ('너무너무너무', 0.9999606090273322),
 ('신용재', 0.998991194480233),
 ('오블리스', 0.9989863725521622),
 ('갓세븐', 0.9982710816259988),
 ('다비치', 0.9981432513884275),
 ('엠카운트다운', 0.9978307775631691),
 ('세븐', 0.9978210421997837),
 ('박진영', 0.9975836317984187),
 ('완전체', 0.9973594469004617),
 ('선의', 0.9963128215975839),
 ('산들', 0.9958319893090414),
 ('중독성', 0.9948644479894773),
 ('프로듀스101', 0.9946890725030576),
 ('열창', 0.9938200380735884),
 ('펜타곤', 0.9934422266805437),
 ('잠깐', 0.9929667382454291),
 ('상큼', 0.9909673401797572),
 ('소녀들', 0.9908127932033489),
 ('엠넷', 0.9907514986652862),
 ('걸크러쉬', 0.99017203825805),
 ('일산동구', 0.9884164745297143),
 ('음악방송', 0.9881439461828352),
 ('사나', 0.9880894585465715),
 ('선율', 0.9875086307696628),
 ('타이틀곡', 0.9869906112674688),
 ('코드', 0.9867835556082788),
 ('본명', 0.98596911773225),
 ('깜찍', 0.9853881990008125)]


In [16]:
pprint(proportion_ratio_keyword('최순실', x, min_count=100)[:30])

[('최순실', 1.0),
 ('게이트', 0.9981018054860111),
 ('정유라', 0.9949748004314443),
 ('연설문', 0.9900718598746623),
 ('모녀', 0.9875768099004291),
 ('승마', 0.9872307511905503),
 ('개명', 0.986641026457457),
 ('비선', 0.985018930232134),
 ('더블루케이', 0.9838995868457685),
 ('실세', 0.9823312201845503),
 ('스포츠재단', 0.980984809482314),
 ('최씨', 0.9802224596736517),
 ('최경희', 0.980172024643097),
 ('비덱', 0.9794924174652362),
 ('이화여대', 0.9792281858488985),
 ('특혜', 0.9775213977582151),
 ('미르재단', 0.9774516345256685),
 ('의혹들', 0.977198367560925),
 ('학점', 0.976567846725211),
 ('비선실세', 0.9747618098586102),
 ('이대', 0.9713049096885505),
 ('미르', 0.9697354303371427),
 ('재단', 0.9665692895878129),
 ('정유라씨', 0.9651208193465403),
 ('엄정', 0.9635099910913556),
 ('차은택', 0.9630949366283257),
 ('이화', 0.962975945484486),
 ('국정조사', 0.9614360445588696),
 ('사퇴', 0.961117249105005),
 ('의혹', 0.9610013059869946)]
